In [4]:
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from bs4 import BeautifulSoup
import re

In [5]:
def aceptar_cookies(año, liga, jornada = None):
    '''
    Inicia el driver de un año, liga y jornada que accede
    al código de la página para extraer los datos

    Parameters
    ----------
    año: int
        Año de los datos a extraer
    liga: str
        Nombre de la liga de los datos a extraer
    jornada: int
        Número de la jornada a partir de la cual empezar la búsqueda
        En caso de no especificarlo, el driver a la página principal
        la cual corresponde a la última jornada de ese año
    
    Returns
    -------
    driver: webdriver
        Elemento webdriver con los datos de la página con los datos del 
        año, liga y jornada. Se usará este driver para extraer el código html
    '''
    
    driver_dir = 'chrome_driver/chromedriver.exe'
    driver=webdriver.Chrome(driver_dir)
    if jornada:
        driver.get("https://www.resultados-futbol.com/"+liga+str(año)+"/grupo1/jornada"+str(jornada))
    else:
        driver.get("https://www.resultados-futbol.com/"+liga+str(año))

    accept_cookies = driver.find_elements_by_xpath('//button[@class="sc-ifAKCX hYNOwJ"]')
    
    try:
        for button in accept_cookies:
            if button.text == "ACEPTO":
                relevant_button = button
                relevant_button.click()
    except:
        pass
    finally:
        return driver

In [6]:
def extraer_datos_jornada(soup):
    '''
    Devuelve los datos de la tabla de posiciones

    Parameters
    ----------
    soup: BeautifulSoup
        Contiene toda la información del código html de la 
        página con los datos sobre el año, liga y jornada

    Returns
    -------
    tabla_jornada: list
        Si el scraping ha salido bien, devuelve una lista de listas con:
        Posicion, Equipo, Puntos, Jornada, Partidos ganados, Partidos empatados,
        Partidos perdidos, Goles a favor, y goles en contra
        En caso de que haya habido algún problema, tabla_jornada se convierte en 
        una lista de valores nulos
    '''
    
    if soup.find("table", {"id": 'tabla2'}):
        table_soup = soup.find("table", {"id": 'tabla2'}).find('tbody').find_all('tr')
    else:
        return None
    num_equipos = len(table_soup)

    Posicion = [table_soup[i].find('th').text for i in range(num_equipos)]
    Equipo = [table_soup[i].find('td', {'class':'equipo'}).find('a').text for i in range(num_equipos)]
    Puntos = [table_soup[i].find('td', {'class':'pts'}).text for i in range(num_equipos)]
    Jornada = [table_soup[i].find('td', {'class':'pj'}).text for i in range(num_equipos)]
    Ganados = [table_soup[i].find('td', {'class':'win'}).text for i in range(num_equipos)]
    Empatados = [table_soup[i].find('td', {'class':'draw'}).text for i in range(num_equipos)]
    Perdidos = [table_soup[i].find('td', {'class':'lose'}).text for i in range(num_equipos)]
    Goles_a_favor= [table_soup[i].find('td', {'class':'f'}).text for i in range(num_equipos)]
    Goles_en_contra= [table_soup[i].find('td', {'class':'c'}).text for i in range(num_equipos)]

    tabla_jornada = [Posicion, Equipo, Puntos, Jornada, Ganados, Empatados, Perdidos, Goles_a_favor, Goles_en_contra]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_jornada])) == 1:
        return tabla_jornada
    else:
        return [None] * len(tabla_jornada)

In [7]:
def extraer_datos_partido(soup):
    '''
    Devuelve los datos de la tabla de partidos

    Parameters
    ----------
    soup: BeautifulSoup
        Contiene toda la información del código html de la 
        página con los datos sobre el año, liga y jornada

    Returns
    -------
    tabla_partido: list
        Si el scraping ha salido bien, devuelve una lista de listas con:
        Nombre del equipo local, nombre del equipo visitante, resultado del partido
        y fecha del partido
        En caso de que haya habido algún problema, tabla_partido se convierte en 
        una lista de valores nulos
    '''

    regex = re.compile('vevent')
    if soup.find("table", {"id": 'tabla1'}):
        table_soup = soup.find("table", {"id": 'tabla1'}).find('tbody').find_all("tr", {"class" : regex})
    else:
        return None
    
    num_partidos = len(table_soup)
    Equipo_Local = [table_soup[i].find('td', {'class':'equipo1'}).find_all('a')[1].text for i in range(num_partidos)]
    Equipo_Visitante = [table_soup[i].find('td', {'class':'equipo2'}).find_all('a')[1].text for i in range(num_partidos)]
    Resultado = [table_soup[i].find('td', {'class':'rstd'}).find('a', {'class':'url'}).find('span').text for i in range(num_partidos)]
    Fecha = [table_soup[i].find('td', {'class':'fecha'}).text for i in range(num_partidos)]

    tabla_partido = [Equipo_Local, Equipo_Visitante, Resultado, Fecha]

    # Nos aseguramos que ningun dato se ha saltado, y que hemos encontrado todos
    if len(set([len(i) for i in tabla_partido])) == 1:
        return tabla_partido
    else:
        return [None] * len(tabla_partido)

In [13]:
def extraer_num_jornadas(soup):
    '''
    Devuelve el número de jornadas correspondiente a un año y liga en concreto

    Parameters
    ----------
    soup: BeautifulSoup
        Contiene toda la información del código html de la 
        página con los datos sobre el año, liga y jornada

    Returns
    -------
    int
        Si la página tiene información acerca de las jornadas correspondiente a ese año
        y liga, devuelve el número de jornadas
        En caso contrario, devuelve 0
    '''

    if soup.find("div", {"id":'desplega_jornadas', "class":'hidden'}) :
        jornadas_soup = soup.find("div", {"id":'desplega_jornadas', "class":'hidden'}).find_all('li')
        return len(jornadas_soup)
    else:
        return 0

In [17]:
def extraccion_datos(año1, año2, *Ligas, primera_jornada = 1):
    '''
    Busca datos sobre posiciones y partidos entre los años entre 
    año1 y año2, para las ligas incluidas en *Ligas
    Actualiza una variable global correspondiente a un diccionario.
    Dicho diccionario obtiene sus keys de una lista con las keys deseadas.
    Una vez actualizado el diccionario, crea un dataframe y lo exporta a
    un archivo excel.

    Parameters
    ----------
    año1: int
        Año a partir del cual empezar la búsqueda
    año2: int
        Año a partir del cual finalizar la búsqueda
    *Ligas: str
        Nombre de las ligas para realizar la búsqueda
    primera_jornada: int, default = 1
        Número de la jornada a partir de la cual empezar la búsqueda

    Returns
    -------
    list
        Lista que contiene el nombre de las ligas de las cuales se ha
        realizado la búsqueda
    DDBB_Resultado : xlsx
        Exporta a un archivo excel los datos sobre los partidos jugados entre el
        año1 y año2 en las ligas *Liga
    DDBB_Tabla : xlsx
        Exporta a un archivo excel los datos sobre los posiciones de los equipos #
        entre el año1 y año2 en las ligas *Liga
    '''
    for liga in Ligas:
        for año in range(año1, año2 + 1):
            print(f'Extrayendo datos del año {año} de la liga {liga}') 
            driver = aceptar_cookies(año, liga) # Accedemos simplemente a la primera jornada de ese año para comprobar cuantas jornadas hubo ese año
            page = driver.page_source
            soup = BeautifulSoup(page, 'html.parser')
            num_jornadas = extraer_num_jornadas(soup)
            if num_jornadas == 0:
                print(f'No hay datos disponibles para el año {año} en la liga {liga}')
                print('Saltando al siguiente año')
                driver.quit()
                continue
            driver.quit()
            for jornada in range(primera_jornada, num_jornadas + 1):
                print(f'\tExtrayendo jornada numero {jornada} del año {año}') 
                driver = aceptar_cookies(año, liga, jornada)
                page = driver.page_source
                soup = BeautifulSoup(page, 'html.parser')
                tabla_jornada = extraer_datos_jornada(soup)
                tabla_partido = extraer_datos_partido(soup)

                if tabla_jornada == None or tabla_partido == None: # Esto significaría que, a pesar de que la página diga que esa jornada tiene datos, en realidad durante ese año no hubo más jornadas. De esa manera, sale del 'for' de las jornadas, y pasa al siguiente año
                    print(f'------------------------------------------------------')
                    print(f'!!! La jornada {jornada} no existe en el año {año} !!!')
                    print(f'!!!            Saltando al siguiente año           !!!')
                    print(f'------------------------------------------------------')
                    driver.quit()
                    break
                
                for i, key in enumerate(list_jornadas[:-2]):
                    dict_jornadas[key].extend(tabla_jornada[i])

                dict_jornadas['Año'].extend([año] * len(tabla_jornada[0]))
                dict_jornadas['Liga'].extend([liga] * len(tabla_jornada[0]))

                for i, key in enumerate(list_partidos[:-3]):
                    dict_partidos[key].extend(tabla_partido[i])

                dict_partidos['Año_partido'].extend([año] * len(tabla_partido[0]))
                dict_partidos['Jornada_partido'].extend([jornada] * len(tabla_partido[0]))
                dict_partidos['Liga_partido'].extend([liga] * len(tabla_partido[0]))
               
                print(f'\tDatos extraídos, quedan {num_jornadas - jornada} jornadas restantes este año')
                driver.quit()
            print(f'Datos del año {año} extraídos, queda(n) {año2 - año} año(s) restante(s)')
            primera_jornada = 1
        
        Data_Tabla = pd.DataFrame(dict_jornadas).drop_duplicates()
        Data_Tabla.to_excel(f"DDBB_Tabla_{año_1}_{año_2}_{liga}.xlsx")
        for key in list_jornadas:
            dict_jornadas[key].clear()

        Data_Partido = pd.DataFrame(dict_partidos).drop_duplicates()
        Data_Partido.columns = ['Equipo_Local', 'Equipo_Visitante', 'Resultado', 'Fecha', 'Año', 'Jornada', 'Liga']
        Data_Partido.to_excel(f"DDBB_Resultados_{año_1}_{año_2}_{liga}.xlsx")
        for key in list_partidos:
            dict_partidos[key].clear()
    return list(Ligas)

In [15]:
list_jornadas = ['Posicion', 'Equipo', 'Puntos', 'Jornada', 'Ganados', 'Empatados', 'Perdidos', 'Goles_a_favor', 'Goles_en_contra', 'Año', 'Liga'] 
list_partidos = ['Equipo_Local', 'Equipo_Visitante', 'Resultado', 'Fecha', 'Año_partido', 'Jornada_partido', 'Liga_partido']
dict_jornadas = {x:[] for x in list_jornadas}
dict_partidos = {x:[] for x in list_partidos}

In [18]:
año_1 = 1990
año_2 = 2020
Ligas = extraccion_datos(año_1, año_2, 'primera', 'segunda', 'premier', 'championship', 'ligue_1', 'ligue_2', 'serie_a', 'serie_b')

SyntaxError: invalid syntax (<ipython-input-18-15514ee6afb8>, line 3)